<a href="https://colab.research.google.com/github/pritish0666/Steganography_using_StyleTransfer/blob/master/FINAL_MAJOR_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#cell Negative
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. SET YOUR FOLDER PATH HERE
# Replace the folder name EXACTLY as it appears in Drive
folder_path = "/content/drive/My Drive/STEGO_MAJOR_PROJECT/scraped_images_640_yolo"

# 3. Check if the folder exists
if os.path.exists(folder_path):
    print("Folder found! ✔")
    print("Path:", folder_path)
    print("Files inside:")
    print(os.listdir(folder_path))
else:
    print("❌ Folder NOT found. Check the path again.")


Mounted at /content/drive
Folder found! ✔
Path: /content/drive/My Drive/STEGO_MAJOR_PROJECT/scraped_images_640_yolo
Files inside:
['data.yaml', 'val', 'train']


In [2]:
import os

DATA_ROOT = "/content/drive/MyDrive/STEGO_MAJOR_PROJECT/scraped_images_640_yolo"

def is_dataset_root(p):
    return os.path.isdir(os.path.join(p,"train","images")) and os.path.isdir(os.path.join(p,"val","images"))

if not os.path.exists(DATA_ROOT):
    raise FileNotFoundError(f"DATA_ROOT not found: {DATA_ROOT}. Upload your folder to Colab and update DATA_ROOT.")

if not is_dataset_root(DATA_ROOT):
    raise RuntimeError(f"Data root missing required train/images & val/images folders: {DATA_ROOT}")

print("✅ DATA_ROOT is set to:", DATA_ROOT)
print("Top-level in DATA_ROOT:", os.listdir(DATA_ROOT))
print("train/images count:", len(os.listdir(os.path.join(DATA_ROOT,"train","images"))))
print("train/labels count:", len(os.listdir(os.path.join(DATA_ROOT,"train","labels"))))


✅ DATA_ROOT is set to: /content/drive/MyDrive/STEGO_MAJOR_PROJECT/scraped_images_640_yolo
Top-level in DATA_ROOT: ['data.yaml', 'val', 'train']
train/images count: 464
train/labels count: 464


In [3]:
import os, math, random, time
from glob import glob
from collections import defaultdict
from PIL import Image, ImageDraw
import torch, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torchvision import models, transforms

OUTPUTS_DIR = "/content/outputs"
os.makedirs(OUTPUTS_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [4]:
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, "train", "images")
TRAIN_LBL_DIR = os.path.join(DATA_ROOT, "train", "labels")

def img_for_label(lbl_path):
    stem = os.path.splitext(os.path.basename(lbl_path))[0]
    for ext in (".jpg", ".jpeg", ".png", ".bmp", ".webp"):
        p = os.path.join(TRAIN_IMG_DIR, stem + ext)
        if os.path.exists(p):
            return p
    return None

class_to_entries = defaultdict(list)
lbl_files = sorted(glob(os.path.join(TRAIN_LBL_DIR, "*.txt")))
print("Found label files:", len(lbl_files))

for lf in lbl_files:
    img_path = img_for_label(lf)
    if not img_path:
        continue
    W, H = Image.open(img_path).size
    boxes_by_class = defaultdict(list)
    with open(lf, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5: continue
            cid = int(parts[0])
            cx, cy, w, h = map(float, parts[1:5])
            xmin = max(0, int((cx - w/2) * W))
            ymin = max(0, int((cy - h/2) * H))
            xmax = min(W, int((cx + w/2) * W))
            ymax = min(H, int((cy + h/2) * H))
            if xmax > xmin and ymax > ymin:
                boxes_by_class[cid].append((xmin, ymin, xmax, ymax))
    for cid, bboxes in boxes_by_class.items():
        class_to_entries[cid].append((img_path, bboxes))

for cid in range(16):
    print(f"class {cid:2d} -> {len(class_to_entries[cid])} entries")

missing = [cid for cid in range(16) if len(class_to_entries[cid]) == 0]
if missing:
    print("WARNING: missing classes:", missing)


Found label files: 464
class  0 -> 27 entries
class  1 -> 28 entries
class  2 -> 31 entries
class  3 -> 29 entries
class  4 -> 28 entries
class  5 -> 28 entries
class  6 -> 28 entries
class  7 -> 30 entries
class  8 -> 28 entries
class  9 -> 28 entries
class 10 -> 28 entries
class 11 -> 32 entries
class 12 -> 29 entries
class 13 -> 30 entries
class 14 -> 30 entries
class 15 -> 30 entries


In [5]:
hex_digits = [hex(i)[2:].upper() for i in range(16)]
hex_to_cid = {h:i for i,h in enumerate(hex_digits)}

def load_tile_for_class(cid, tile_size=128):
    entries = class_to_entries.get(cid, [])
    if entries:
        img_path, bboxes = random.choice(entries)
        img = Image.open(img_path).convert("RGB")
        if bboxes:
            xmin,ymin,xmax,ymax = random.choice(bboxes)
            tile = img.crop((xmin,ymin,xmax,ymax))
        else:
            W,H = img.size
            side = min(W,H); left=(W-side)//2; top=(H-side)//2
            tile = img.crop((left,top,left+side,top+side))
        return tile.resize((tile_size,tile_size), Image.BICUBIC)
    all_imgs = sorted(glob(os.path.join(TRAIN_IMG_DIR, "*.*")))
    if all_imgs:
        img = Image.open(random.choice(all_imgs)).convert("RGB")
        W,H = img.size; side=min(W,H); left=(W-side)//2; top=(H-side)//2
        return img.crop((left,top,left+side,top+side)).resize((tile_size,tile_size), Image.BICUBIC)
    t = Image.new("RGB",(tile_size,tile_size),(240,240,240)); ImageDraw.Draw(t).text((8,8), str(cid), fill=(0,0,0)); return t

def text_to_hex(text):
    return "".join(f"{ord(c):02X}" for c in text)

def make_grid_from_hex(hex_str, grid_size=None, tile_size=128):
    digits = list(hex_str)
    if grid_size is None:
        grid_size = math.ceil(math.sqrt(len(digits)))
    total = grid_size*grid_size
    if len(digits) < total:
        digits += ['0']*(total-len(digits))
    canvas = Image.new("RGB",(grid_size*tile_size, grid_size*tile_size),(255,255,255))
    for idx,d in enumerate(digits[:total]):
        r,c = divmod(idx, grid_size)
        cid = hex_to_cid.get(d.upper(),0)
        tile = load_tile_for_class(cid, tile_size=tile_size)
        canvas.paste(tile, (c*tile_size, r*tile_size))
    return canvas

def secret_image_to_hex(secret_img_path, resize=(16,16)):
    img = Image.open(secret_img_path).convert("RGB")
    img_small = img.resize(resize, Image.BICUBIC)
    data = img_small.tobytes()
    hex_str = "".join(f"{b:02X}" for b in data)
    return hex_str, img_small

def make_dataset_grid_from_secret_image(secret_img_path, resize=(16,16), tile_size=64, grid_size=None):
    hex_str, small = secret_image_to_hex(secret_img_path, resize=resize)
    grid = make_grid_from_hex(hex_str, grid_size=grid_size, tile_size=tile_size)
    return grid, hex_str, small


In [6]:
def replace_relu_with_outplace(module):
    for name, layer in module.named_children():
        if isinstance(layer, nn.ReLU):
            setattr(module, name, nn.ReLU(inplace=False))
        else:
            replace_relu_with_outplace(layer)

cnn = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features.to(device).eval()
replace_relu_with_outplace(cnn)

def image_loader(path, imsize=256):
    loader = transforms.Compose([transforms.Resize((imsize,imsize)), transforms.ToTensor()])
    img = Image.open(path).convert("RGB")
    t = loader(img).unsqueeze(0).to(device, torch.float)
    return t

def im_to_pil(tensor):
    t = tensor.clone().detach().cpu().squeeze(0)
    return transforms.ToPILImage()(torch.clamp(t,0,1))

def gram_matrix(x):
    b,ch,h,w = x.size()
    f = x.view(b,ch,h*w)
    return torch.bmm(f, f.transpose(1,2)) / (ch*h*w)

class ContentLoss(nn.Module):
    def __init__(self, target):
        super().__init__()
        self.target = target.detach(); self.loss = 0
    def forward(self,x):
        self.loss = F.mse_loss(x, self.target); return x

class StyleLoss(nn.Module):
    def __init__(self,target):
        super().__init__()
        self.target = gram_matrix(target).detach(); self.loss = 0
    def forward(self,x):
        self.loss = F.mse_loss(gram_matrix(x), self.target); return x

def get_model_and_losses(cnn, style_img, content_img):
    content_layers = ['21']
    style_layers = ['0','5','10','19','28']
    model = nn.Sequential().to(device)
    style_losses, content_losses = [], []
    i=0
    for layer in cnn.children():
        model.add_module(str(i), layer)
        if str(i) in style_layers:
            target = model(style_img).detach(); sl = StyleLoss(target); model.add_module("style_loss_"+str(i), sl); style_losses.append(sl)
        if str(i) in content_layers:
            target = model(content_img).detach(); cl = ContentLoss(target); model.add_module("content_loss_"+str(i), cl); content_losses.append(cl)
        i+=1
    return model, style_losses, content_losses

def generate_stego(secret_path, cover_path, out_path, imsize=256, alpha=0.1, beta=4e4, steps=200):
    content_img = image_loader(secret_path, imsize)
    style_img = image_loader(cover_path, imsize)
    input_img = style_img.clone()
    model, style_losses, content_losses = get_model_and_losses(cnn, style_img, content_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    run=[0]
    while run[0] <= steps:
        def closure():
            optimizer.zero_grad()
            input_img.data.clamp_(0,1)
            model(input_img)
            style_score = sum(sl.loss for sl in style_losses)
            content_score = sum(cl.loss for cl in content_losses)
            loss = alpha*content_score + beta*style_score
            loss.backward()
            run[0]+=1
            if run[0] % 20 == 0 or run[0]==steps:
                print(f"Step {run[0]} | Content {content_score.item():.4e} | Style {style_score.item():.4e}")
            return loss
        optimizer.step(closure)
    input_img.data.clamp_(0,1)
    out = im_to_pil(input_img); out.save(out_path)
    print("✅ Saved stego to:", out_path)
    return out


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:07<00:00, 79.9MB/s]


In [7]:
# # === Cell 5 (corrected) — Main interactive flow (saves uploaded files to OUTPUTS_DIR) ===
# import os, time, shutil
# from google.colab import files

# print("Choose mode:\n  1) Text → Stego (text uses dataset tiles)\n  2) Image → Stego (secret image -> hex -> dataset tiles)\n")
# mode = input("Enter 1 or 2: ").strip()

# # Ensure outputs dir exists
# os.makedirs(OUTPUTS_DIR, exist_ok=True)

# if mode == "1":
#     text = input("Enter text to hide (keep reasonably short): ").strip()
#     grid_size_input = input("Grid size (enter for auto): ").strip()
#     tile_size = int(input("Tile size in px (recommend 64 or 128): ").strip() or "128")
#     grid_size = int(grid_size_input) if grid_size_input else None

import os, time, shutil
from google.colab import files

print("Choose mode:\n  1) Text → Stego\n  2) Image → Stego\n")
mode = input("Enter 1 or 2: ").strip()

os.makedirs(OUTPUTS_DIR, exist_ok=True)

tile_size = 64
default_resize = (16,16)
default_steps = 40
default_imsize = 256

if mode == "1":
    text = input("Enter text to hide (keep reasonably short): ").strip()
    grid_size_input = input("Grid size (enter for auto): ").strip()
    grid_size = int(grid_size_input) if grid_size_input else None
    secret_grid = make_grid_from_hex(text_to_hex(text), grid_size=grid_size, tile_size=tile_size)
    secret_basename = f"secret_text_grid_{int(time.time())}.png"
    secret_path = os.path.join(OUTPUTS_DIR, secret_basename)
    secret_grid.save(secret_path)
    print("Secret grid saved:", secret_path)

elif mode == "2":
    print("Upload SECRET image file now (it will be used as secret content):")
    uploaded = files.upload()
    sec_fp = list(uploaded.keys())[0]
    secret_input_path = os.path.join("/content", sec_fp)
    saved_secret_fp = f"secret_uploaded_{int(time.time())}_{sec_fp}"
    saved_secret_path = os.path.join(OUTPUTS_DIR, saved_secret_fp)
    shutil.copy2(secret_input_path, saved_secret_path)
    print("Saved a copy of the uploaded secret to:", saved_secret_path)

    grid_size_input = input("Grid size (enter for auto): ").strip()
    grid_size = int(grid_size_input) if grid_size_input else None

    print("Converting secret image -> hex -> dataset grid...")
    secret_grid, hex_str, small_preview = make_dataset_grid_from_secret_image(
        secret_input_path, resize=default_resize, tile_size=tile_size, grid_size=grid_size
    )

    secret_basename = f"secret_image_grid_{int(time.time())}.png"
    secret_path = os.path.join(OUTPUTS_DIR, secret_basename)
    secret_grid.save(secret_path)

    preview_basename = f"secret_small_preview_{int(time.time())}.png"
    preview_path = os.path.join(OUTPUTS_DIR, preview_basename)
    small_preview.save(preview_path)

    print("Secret grid saved:", secret_path)
    print("Small preview saved:", preview_path)

else:
    raise ValueError("Invalid mode selected. Enter '1' or '2'")

print("\nUpload COVER image now (it will be used as cover/style):")
uploaded = files.upload()
cover_fp = list(uploaded.keys())[0]
cover_temp_path = os.path.join("/content", cover_fp)
cover_saved_name = f"cover_{int(time.time())}_{cover_fp}"
cover_path = os.path.join(OUTPUTS_DIR, cover_saved_name)
shutil.copy2(cover_temp_path, cover_path)
print("Saved cover copy to:", cover_path)

alpha = float(input("Content weight alpha (default 0.1): ").strip() or "0.1")
beta = float(input("Style weight beta (default 40000): ").strip() or "40000")

out_name = f"stego_{int(time.time())}.png"
out_path = os.path.join(OUTPUTS_DIR, out_name)
print("\nRunning style-transfer embedding...\n")

stego_img = generate_stego(secret_path, cover_path, out_path, imsize=default_imsize, alpha=alpha, beta=beta, steps=default_steps)

print("\nDone. Stego saved at:", out_path)
print("All related files (secret, cover, preview) are kept in:", OUTPUTS_DIR)



Choose mode:
  1) Text → Stego
  2) Image → Stego

Enter 1 or 2: 2
Upload SECRET image file now (it will be used as secret content):


Saving mountain.jpg to mountain.jpg
Saved a copy of the uploaded secret to: /content/outputs/secret_uploaded_1763867345_mountain.jpg
Grid size (enter for auto): 32
Converting secret image -> hex -> dataset grid...
Secret grid saved: /content/outputs/secret_image_grid_1763867360.png
Small preview saved: /content/outputs/secret_small_preview_1763867362.png

Upload COVER image now (it will be used as cover/style):


Saving snow.jpg to snow.jpg
Saved cover copy to: /content/outputs/cover_1763867372_snow.jpg
Content weight alpha (default 0.1): 
Style weight beta (default 40000): 

Running style-transfer embedding...

Step 20 | Content 4.9389e+01 | Style 9.0070e-06
Step 40 | Content 4.3083e+01 | Style 1.2250e-05
Step 60 | Content 4.0628e+01 | Style 1.3027e-05
✅ Saved stego to: /content/outputs/stego_1763867375.png

Done. Stego saved at: /content/outputs/stego_1763867375.png
All related files (secret, cover, preview) are kept in: /content/outputs


In [8]:
1# Cell 6 — Save outputs to Google Drive (optional)
save_drive = input("Save outputs to Google Drive? (y/n): ").strip().lower()
if save_drive == "y":
    from google.colab import drive, files
    drive.mount('/content/drive')
    DRIVE_DIR = "/content/drive/MyDrive/StegoOutputs"
    os.makedirs(DRIVE_DIR, exist_ok=True)
    # copy all files in OUTPUTS_DIR
    import shutil
    for fn in os.listdir(OUTPUTS_DIR):
        src = os.path.join(OUTPUTS_DIR, fn)
        dst = os.path.join(DRIVE_DIR, fn)
        shutil.copy2(src, dst)
    print("Saved outputs to:", DRIVE_DIR)
    print("Listing saved files:")
    print(os.listdir(DRIVE_DIR)[-20:])
else:
    print("Skipped saving to Drive.")


Save outputs to Google Drive? (y/n): n
Skipped saving to Drive.


In [9]:
!pip install scikit-image lpips


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00


In [11]:
import os, cv2, numpy as np, torch, lpips
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

cover_path = "/content/outputs/cover_1763867372_snow.jpg"
stego_path = "/content/outputs/stego_1763867375.png"
cover_bgr = cv2.imread(cover_path, cv2.IMREAD_UNCHANGED)
stego_bgr = cv2.imread(stego_path, cv2.IMREAD_UNCHANGED)

if cover_bgr is None:
    raise FileNotFoundError(f"Cover not found: {cover_path}")
if stego_bgr is None:
    raise FileNotFoundError(f"Stego not found: {stego_path}")

if cover_bgr.ndim == 3 and cover_bgr.shape[2] == 4:
    cover_bgr = cv2.cvtColor(cover_bgr, cv2.COLOR_BGRA2BGR)
if stego_bgr.ndim == 3 and stego_bgr.shape[2] == 4:
    stego_bgr = cv2.cvtColor(stego_bgr, cv2.COLOR_BGRA2BGR)

cover_rgb = cv2.cvtColor(cover_bgr, cv2.COLOR_BGR2RGB)
stego_rgb  = cv2.cvtColor(stego_bgr, cv2.COLOR_BGR2RGB)

Hc, Wc = cover_rgb.shape[:2]
Hs, Ws = stego_rgb.shape[:2]
if (Hc, Wc) != (Hs, Ws):
    stego_rgb = cv2.resize(stego_rgb, (Wc, Hc), interpolation=cv2.INTER_AREA)

MIN_SIDE = 64
Hc, Wc = cover_rgb.shape[:2]
if min(Hc, Wc) < MIN_SIDE:
    new_w = max(MIN_SIDE, Wc)
    new_h = max(MIN_SIDE, Hc)
    cover_rgb = cv2.resize(cover_rgb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    stego_rgb = cv2.resize(stego_rgb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    Hc, Wc = new_h, new_w
    print(f"Upsampled images to ({Hc},{Wc}) to satisfy SSIM window size requirements.")

cover_arr = np.asarray(cover_rgb).astype(np.float32)
stego_arr  = np.asarray(stego_rgb).astype(np.float32)

try:
    ssim_score = ssim(cover_arr, stego_arr, data_range=255.0, channel_axis=2)
except Exception as e:
    print("SSIM failed on RGB with error:", e)
    cover_gray = cv2.cvtColor(cover_rgb, cv2.COLOR_RGB2GRAY)
    stego_gray = cv2.cvtColor(stego_rgb, cv2.COLOR_RGB2GRAY)
    ssim_score = ssim(cover_gray, stego_gray, data_range=255.0)

psnr_score = psnr(cover_arr, stego_arr, data_range=255.0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = lpips.LPIPS(net='alex').to(device)

def to_lpips_tensor(img_rgb):
    t = torch.from_numpy(img_rgb.astype(np.float32) / 255.0).permute(2,0,1).unsqueeze(0)
    t = t * 2.0 - 1.0
    return t.to(device)

with torch.no_grad():
    a = to_lpips_tensor(cover_rgb)
    b = to_lpips_tensor(stego_rgb)
    lpips_score = loss_fn(a, b).item()

print("Image size used for metrics (HxW):", (Hc, Wc))
print("                     ")
print(f"SSIM:  {ssim_score:.4f}   (1.0 = identical)")
print(f"PSNR:  {psnr_score:.2f} dB")
print(f"LPIPS: {lpips_score:.4f}   (0 = perceptually identical)")

# print("\nGuidance: SSIM >0.9 & PSNR >35 dB & LPIPS <0.2 -> very high visual similarity.")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 175MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/alex.pth
Image size used for metrics (HxW): (523, 860)
                     
SSIM:  0.7638   (1.0 = identical)
PSNR:  31.24 dB
LPIPS: 0.5447   (0 = perceptually identical)
